## Imports

In [36]:
!pip install transformers torch torchvision Pillow
!pip install open_clip_torch

In [37]:
import pandas as pd
import os
from PIL import Image

In [38]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [39]:
# à modifier selon votre chemin

base_path = "/content/drive/MyDrive/ISIS/5A/PTUT Ceveop/Ressources/"

csv_physique = base_path + "inventaire_physique.csv"
csv_comptable = base_path + "inventaire_comptable.csv"
img_folder = base_path + "Images/"

## Préparation des données

1/ Inventaire physique réduit => columns : Img/Titre/codeEtiquette/NumInventaire

2/ Inventaire comptable => NumInventaire/des1/fournisseur

3/ lien entre les 2 (numInventaire) => correspondances réelles

In [40]:
df_physique = pd.read_csv(csv_physique)
df_comptable = pd.read_csv(csv_comptable,sep=";",encoding="latin-1")

/tmp/ipython-input-3699852166.py:1: DtypeWarning: Columns (1,9,12) have mixed types. Specify dtype option on import or set low_memory=False.
  df_physique = pd.read_csv(csv_physique)


In [41]:
print(df_physique.columns.tolist())
print(df_comptable.columns.tolist())


['Titre', 'Code Étiquette', 'Date Création', 'Catégorie Parent', 'Sous Catégorie', 'Localisation', 'UF', 'Statut', 'Num Produit', 'Description', 'Img', 'Etat', 'Date modif statut', 'Num Fiche Amortissement', 'Num Commande', 'Num Inventaire', 'Fournisseur']
['numFicheAmortissement', 'des1', 'des2', 'quantite', 'mtOrigine', 'numInventaire', 'dateReception', 'dureeFiche', 'dateFin', 'uf', 'libelleUf', 'numCommande', 'fournisseur']


In [42]:
# Harmonisation du NumInventaire
df_physique = df_physique.rename(columns={"Num Inventaire": "NumInventaire"})
df_comptable = df_comptable.rename(columns={"numInventaire": "NumInventaire"})

# Supprime les lignes sans identifiant
df_physique = df_physique.dropna(subset=["NumInventaire"])
df_comptable = df_comptable.dropna(subset=["NumInventaire"])

In [43]:
# Création df commun
df_merged = pd.merge(
    df_physique,
    df_comptable,
    on="NumInventaire",
    how="inner"
)

print(len(df_merged), "correspondances trouvées dans le dataset")

6894 correspondances trouvées dans le dataset


In [44]:
# Création d’un texte descriptif pour chaque fiche comptable
df_merged["fiche_textuelle"] = (
    df_merged["des1"].fillna('') + " " +
    df_merged["des2"].fillna('') + " " +
    df_merged["fournisseur"].fillna('')
).str.strip()

# Sélection des colonnes importantes
df_final = df_merged[[
    "Titre",
    "Img",
    "NumInventaire",
    "fiche_textuelle"
]]

print(df_final.head(3))

    Titre                                       Img NumInventaire  \
0  Chaise  b7c180e6-13d0-4365-afe7-c83a25746d8d.jpg     202000166   
1  Chaise  c88762c5-3c03-4ad2-8a8a-b02add20dc4d.jpg     202000166   
2  Chaise  63474d32-c1a6-44c1-b3f5-62bdcc2f8d24.jpg       9616256   

                                     fiche_textuelle  
0      CHAISE APPUI SUR TABLE LOLA REF. 2748678 UGAP  
1      CHAISE APPUI SUR TABLE LOLA REF. 2748678 UGAP  
2  CHAISE EMPILABLE 250 ASSISE ET DOSSIER BOIS CO...  


In [45]:
# réduction à 100 lignes aléatoires
df_test_sample = df_final.sample(n=100, random_state=42).reset_index(drop=True)

## Initialisation du modèle

In [54]:
from transformers import BlipProcessor, BlipForImageTextRetrieval
from PIL import Image
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"

# Charger le modèle spécialisé
processor = BlipProcessor.from_pretrained("Salesforce/blip-itm-base-coco")
model = BlipForImageTextRetrieval.from_pretrained("Salesforce/blip-itm-base-coco").to(device)
model.eval()

BlipForImageTextRetrieval(
  (vision_model): BlipVisionModel(
    (embeddings): BlipVisionEmbeddings(
      (patch_embedding): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
    )
    (encoder): BlipEncoder(
      (layers): ModuleList(
        (0-11): 12 x BlipEncoderLayer(
          (self_attn): BlipAttention(
            (dropout): Dropout(p=0.0, inplace=False)
            (qkv): Linear(in_features=768, out_features=2304, bias=True)
            (projection): Linear(in_features=768, out_features=768, bias=True)
          )
          (layer_norm1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (mlp): BlipMLP(
            (activation_fn): GELUActivation()
            (fc1): Linear(in_features=768, out_features=3072, bias=True)
            (fc2): Linear(in_features=3072, out_features=768, bias=True)
          )
          (layer_norm2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        )
      )
    )
    (post_layernorm): LayerNorm((768,), eps=1e-05, 

## Génération des embeddings

In [56]:
import numpy as np
from tqdm import tqdm

In [58]:
image_embs = []
text_embs = []

for i, row in tqdm(df_test_sample.iterrows(), total=len(df_test_sample), desc="🔹 Génération des embeddings"):
    try:
        # === IMAGE ===
        img_path = os.path.join(img_folder, row["Img"])
        image = Image.open(img_path).convert("RGB")
        inputs_img = processor(images=image, return_tensors="pt").to(device)

        with torch.no_grad():
            vision_outputs = model.visual_encoder(**inputs_img)
            img_emb = vision_outputs.last_hidden_state.mean(dim=1)  # moyenne des features spatiaux

        image_embs.append(img_emb.cpu().numpy())

        # === TEXTE ===
        fiche = row["fiche_textuelle"]
        inputs_txt = processor(text=[fiche], return_tensors="pt", padding=True, truncation=True).to(device)

        with torch.no_grad():
            txt_outputs = model.text_encoder(**inputs_txt, return_dict=True)
            txt_emb = txt_outputs.last_hidden_state.mean(dim=1)

        text_embs.append(txt_emb.cpu().numpy())

    except Exception as e:
        print(f"⚠️ Erreur ligne {i} ({row['Img']}) : {e}")
        image_embs.append(np.zeros((1, 768)))
        text_embs.append(np.zeros((1, 768)))


🔹 Génération des embeddings:   7%|▋         | 7/100 [00:00<00:01, 63.47it/s]

⚠️ Erreur ligne 0 (2ad95358-27ef-4ea4-92cb-a9b43be3e602.jpg) : 'BlipForImageTextRetrieval' object has no attribute 'visual_encoder'
⚠️ Erreur ligne 1 (acce2bca-ce85-43c7-bc93-2f47daed1a9b.jpeg) : 'BlipForImageTextRetrieval' object has no attribute 'visual_encoder'
⚠️ Erreur ligne 2 (dde6af37-c9c1-48c8-bfb2-1ddb86bef67d.jpg) : 'BlipForImageTextRetrieval' object has no attribute 'visual_encoder'
⚠️ Erreur ligne 3 (b93bc2e2-a648-4d3d-b293-61ad5b4733c2.jpg) : 'BlipForImageTextRetrieval' object has no attribute 'visual_encoder'
⚠️ Erreur ligne 4 (0ec0d22d-8088-402b-8bf2-5c1c4fc2d607.jpg) : 'BlipForImageTextRetrieval' object has no attribute 'visual_encoder'
⚠️ Erreur ligne 5 (1b0eb089-732e-4f42-baef-ad1c7365f78f.jpg) : 'BlipForImageTextRetrieval' object has no attribute 'visual_encoder'
⚠️ Erreur ligne 6 (913e9660-9058-4440-aa2c-4f929cb444e8.jpg) : 'BlipForImageTextRetrieval' object has no attribute 'visual_encoder'
⚠️ Erreur ligne 7 (97f22183-954d-4f0f-9310-67de240e4030.jpeg) : 'BlipForIma

🔹 Génération des embeddings:  21%|██        | 21/100 [00:00<00:01, 61.22it/s]

⚠️ Erreur ligne 13 (548c94d4-0bd5-4c6d-bd68-f201356bdc1a.jpg) : 'BlipForImageTextRetrieval' object has no attribute 'visual_encoder'
⚠️ Erreur ligne 14 (c6e9a622-1e99-4bcf-9153-be59f524b94a.jpg) : 'BlipForImageTextRetrieval' object has no attribute 'visual_encoder'
⚠️ Erreur ligne 15 (24b77188-67fb-45a5-a609-b599cce537d9.jpg) : 'BlipForImageTextRetrieval' object has no attribute 'visual_encoder'
⚠️ Erreur ligne 16 (4071898f-eabd-4793-a104-fdbed752f1cf.jpg) : 'BlipForImageTextRetrieval' object has no attribute 'visual_encoder'
⚠️ Erreur ligne 17 (ea11bfe5-50bf-4cf2-9791-77e797890008.jpg) : 'BlipForImageTextRetrieval' object has no attribute 'visual_encoder'
⚠️ Erreur ligne 18 (5f08b559-d645-41b3-91df-2486d7fe078a.jpg) : 'BlipForImageTextRetrieval' object has no attribute 'visual_encoder'
⚠️ Erreur ligne 19 (3b1d0c7b-4a4b-4431-a00b-5564f824ea3d.jpg) : 'BlipForImageTextRetrieval' object has no attribute 'visual_encoder'
⚠️ Erreur ligne 20 (92ff5ef4-201d-4440-987e-8b5f4d30c282.jpg) : 'Blip

🔹 Génération des embeddings:  35%|███▌      | 35/100 [00:00<00:01, 59.78it/s]

...

⚠️ Erreur ligne 75 (84757801-b6e8-4705-8448-7c6ed2be453e.jpg) : 'BlipForImageTextRetrieval' object has no attribute 'visual_encoder'
⚠️ Erreur ligne 76 (4eac3004-2978-4adf-96ae-95f08b1cc48a.jpg) : 'BlipForImageTextRetrieval' object has no attribute 'visual_encoder'
⚠️ Erreur ligne 77 (7f0a5098-78d6-4913-8efb-93633eda4b3c.jpeg) : 'BlipForImageTextRetrieval' object has no attribute 'visual_encoder'
⚠️ Erreur ligne 78 (2824d5fe-73c8-415a-9eba-966851f37d7a.jpeg) : 'BlipForImageTextRetrieval' object has no attribute 'visual_encoder'
⚠️ Erreur ligne 79 (292b9232-5aee-497c-bc54-425d617636a0.jpg) : 'BlipForImageTextRetrieval' object has no attribute 'visual_encoder'
⚠️ Erreur ligne 80 (08aced95-f496-4efb-b566-0c79029ad697.jpg) : 'BlipForImageTextRetrieval' object has no attribute 'visual_encoder'
⚠️ Erreur ligne 81 (abba4760-c43e-45f9-9b44-f803fbf825d5.jpeg) : 'BlipForImageTextRetrieval' object has no attribute 'visual_encoder'
⚠️ Erreur ligne 82 (7024078d-c04a-4ff7-95d9-eb8011a48f42.jpg) : 'B

🔹 Génération des embeddings:  97%|█████████▋| 97/100 [00:01<00:00, 58.46it/s]

⚠️ Erreur ligne 87 (1662302e-f17f-4637-a179-6c9ce7261002.jpg) : 'BlipForImageTextRetrieval' object has no attribute 'visual_encoder'
⚠️ Erreur ligne 88 (7f216d7d-79e3-433f-8e38-feac66730523.jpg) : 'BlipForImageTextRetrieval' object has no attribute 'visual_encoder'
⚠️ Erreur ligne 89 (8cc1f12b-0bd8-4b0d-b7f9-99f964d32f5a.jpg) : 'BlipForImageTextRetrieval' object has no attribute 'visual_encoder'
⚠️ Erreur ligne 90 (ca906f9a-ae85-4d81-b238-4ee8591131be.jpg) : 'BlipForImageTextRetrieval' object has no attribute 'visual_encoder'
⚠️ Erreur ligne 91 (11f0774e-651c-4cb6-9152-e45021220313.jpg) : 'BlipForImageTextRetrieval' object has no attribute 'visual_encoder'
⚠️ Erreur ligne 92 (eb70ed03-e347-4163-965e-9c9a598dc916.jpg) : 'BlipForImageTextRetrieval' object has no attribute 'visual_encoder'
⚠️ Erreur ligne 93 (a89105b9-4b0d-40ea-b406-6047d713701f.jpg) : 'BlipForImageTextRetrieval' object has no attribute 'visual_encoder'
⚠️ Erreur ligne 94 (cf49aabe-7ed7-42d0-9aab-30a0daf3fd7d.jpg) : 'Blip

🔹 Génération des embeddings: 100%|██████████| 100/100 [00:01<00:00, 59.46it/s]

⚠️ Erreur ligne 99 (277fb895-ce98-4fb9-abd8-a9a50a97c05a.jpg) : 'BlipForImageTextRetrieval' object has no attribute 'visual_encoder'


## Evaluation du modèle